In [1]:
!pwd

/home/jiekaitao/Documents/GitHub/FML_Group_Project


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from sklearn.model_selection import StratifiedKFold

from skimage.transform import rotate

t_train_csv = pd.read_csv('./t_train_project.csv')
x_train_csv = pd.read_csv('./x_train_project.csv')

print("t_train shape:", t_train_csv.shape)
print("x_train shape:", x_train_csv.shape)

X = np.array(x_train_csv).reshape(-1, 100, 100, 1) # the data dimension with dimension "1" needs to be explicitly stated here for the CNN
y = np.array(t_train_csv)

# normalize everything for the CNN and MLP (from 0 to 1)
X_normalized = X / 255.0

X_train_normalized, X_val_normalized, y_train, y_val = train_test_split(X_normalized, y, test_size=0.2, random_state=0, stratify=y)

# print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

2025-04-06 23:11:39.075587: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 23:11:39.085604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743995499.098104 1074554 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743995499.101480 1074554 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-06 23:11:39.115866: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

t_train shape: (5999, 1)
x_train shape: (5999, 10000)


In [3]:
# Without imputed data KFold testing
# ~96.8% accuracy w/ 500 epochs

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

val_accuracies = []
val_losses = []

fold_i = 1
for train_index, val_index in skf.split(X_normalized, y):
    print(f'-=-=-=-= Training Fold {fold_i}/{5} -=-=-=-=')
    X_train_fold, X_val_fold = X_normalized[train_index], X_normalized[val_index]
    y_train_fold, y_val_fold = y[train_index], y[val_index]

    # model = Sequential([
    #     Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    #     MaxPooling2D((2, 2)), # slide a 2x2 pooling filter to reduce overfitting
    #     Conv2D(64, (3, 3), activation='relu'),
    #     MaxPooling2D((2, 2)),
    #     Conv2D(128, (3, 3), activation='relu'),
    #     MaxPooling2D((2, 2)),
    #     Conv2D(256, (3, 3), activation='relu'),
    #     Flatten(), # flatten for the input layer of the MLP
    #     # mlp part
    #     Dense(256, activation='relu'),
    #     Dropout(0.5),
    #     Dense(128, activation='relu'),
    #     Dropout(0.3),
    #     Dense(64, activation='relu'),
    #     Dropout(0.1),
    #     Dense(10, activation='softmax') # output layer for the 10 possible classes
    # ])


    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(512, (3, 3), activation='relu'),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.1),
        Dense(10, activation='softmax')
    ])

    # one-hot encode the labels fro the current fold
    y_train_one_hot_encoded = np.zeros((y_train_fold.shape[0], 10)) # 2d data tensors w/ each row corresponding to one data sample's one hot encoding for calculating the difference (error) between model output and labels
    y_val_one_hot_encoded = np.zeros((y_val_fold.shape[0], 10))

    for i in range(len(y_train_fold)):
        # y_train_fold[i] is shape (1,), so access element with [0]
        y_train_one_hot_encoded[i, y_train_fold[i][0]] = 1

    for i in range(len(y_val_fold)):
         # y_val_fold[i] is shape (1,), so access element with [0]
        y_val_one_hot_encoded[i, y_val_fold[i][0]] = 1

    # print(y_val_one_hot_encoded)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    print(f'-=-=-=-=-=-=- TRAINING FOR FOLD #{fold_i} -=-=-=-=-=-')
    history = model.fit(
        X_train_fold, y_train_one_hot_encoded,
        epochs=16,
        batch_size=128,
        validation_data=(X_val_fold, y_val_one_hot_encoded),
        verbose=0 #reduce the console output
    )

    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_one_hot_encoded, verbose=0)
    print(f"Fold {fold_i} validation accuracy: {val_accuracy}")
    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss)

    fold_i = fold_i + 1

print("\n\nCross-validation summary")
print(f"Score per fold {val_accuracies}")
print(f"Average validation accuracy {np.mean(val_accuracies)}")
print(f"Average validation loss value {np.mean(val_losses)}")

-=-=-=-= Training Fold 1/5 -=-=-=-=


/home/jiekaitao/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1743995505.157742 1074554 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14312 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


-=-=-=-=-=-=- TRAINING FOR FOLD #1 -=-=-=-=-=-


I0000 00:00:1743995506.589001 1074770 service.cc:148] XLA service 0x737d6800fad0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743995506.589024 1074770 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti SUPER, Compute Capability 8.9
2025-04-06 23:11:46.613386: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1743995506.735587 1074770 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1743995510.076100 1074770 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-04-06 23:11:54.145411: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_108_0', 768 bytes spill stores, 720 bytes spill loads

2025-04-06 23:11:54.162619: I 

Fold 1 validation accuracy: 0.9583333134651184
-=-=-=-= Training Fold 2/5 -=-=-=-=
-=-=-=-=-=-=- TRAINING FOR FOLD #2 -=-=-=-=-=-
Fold 2 validation accuracy: 0.9483333230018616
-=-=-=-= Training Fold 3/5 -=-=-=-=
-=-=-=-=-=-=- TRAINING FOR FOLD #3 -=-=-=-=-=-
Fold 3 validation accuracy: 0.9708333611488342
-=-=-=-= Training Fold 4/5 -=-=-=-=
-=-=-=-=-=-=- TRAINING FOR FOLD #4 -=-=-=-=-=-
Fold 4 validation accuracy: 0.9458333253860474
-=-=-=-= Training Fold 5/5 -=-=-=-=
-=-=-=-=-=-=- TRAINING FOR FOLD #5 -=-=-=-=-=-
Fold 5 validation accuracy: 0.9674729108810425


Cross-validation summary
Score per fold [0.9583333134651184, 0.9483333230018616, 0.9708333611488342, 0.9458333253860474, 0.9674729108810425]
Average validation accuracy 0.9581612467765808
Average validation loss value 0.1765238106250763


In [4]:
def vary_intensity(X_normalized, y, proportion=0.2, intensity_min = 0.5, intensity_max = 1.5):
    # proportion = proportion of the X_normalized dataset to impute
    # intensity_min is the minimum possible multiplication factors for the pixel intensity
    # intensity_max is the max possible factor (upper range of gaussian)
    num_samples_to_impute = int(np.floor(proportion*len(X_normalized)))
    indices = np.random.choice(len(X_normalized), num_samples_to_impute, replace=False) # reaplce = False ensures sampels arent picked again (don't put back into possible selection)
    
    X_augmented = []
    y_augmented = []
    
    for i in indices:
        factor = np.random.uniform(intensity_min, intensity_max)
        img_adjusted = np.clip(X_normalized[i] * factor, 0.0, 1.0) # resets to max 1 (normalizes it again)
        X_augmented.append(img_adjusted)
        y_augmented.append(y[i]) # add back in the original label
    # output dimensions are (num_augmented, height, width, channels AKA 1 channel)
    return np.array(X_augmented), np.array(y_augmented)

def vary_rotation(X_normalized, y, proportion=0.2, angle_min=-30, angle_max=30):
    num_samples_to_impute = int(np.floor(proportion*len(X_normalized)))
    indices = np.random.choice(len(X_normalized), num_samples_to_impute, replace=False) # reaplce = False ensures sampels arent picked again (don't put back into possible selection)
    
    X_augmented = []
    y_augmented = []
    
    for i in indices:
        img_2d = X_normalized[i].reshape(100, 100)
        angle = np.random.uniform(angle_min, angle_max)
        # preserve_range True tells skimage to not resize the output image dimensions to fit
        # the full rotated image, so we may **clip some data in the corners!!**
        # mode=edge fills in the new space introduced by the rotation with the pixel values found at the edge
        # preserve_range keeps the pixel values in [0,1] rather than [0,255]
        rotated_img_2d = rotate(img_2d, angle, resize=False, mode='edge', preserve_range=True) # outputs (100,00)
        X_augmented.append(rotated_img_2d.reshape(100, 100, 1)) # need to reshape to add that nested [] pixel intensity value (the dimension with size 1 in the tensor shape)
        y_augmented.append(y[i]) # add back in the original label
    # output dimensions are (num_augmented, height, width, channels AKA 1 channel)
    return np.array(X_augmented), np.array(y_augmented)

# X_intensity, y_intensity = vary_intensity(X_normalized, y)
# X_rotation, y_rotation = vary_rotation(X_normalized, y)

# X_augmented = np.concatenate((X_normalized, X_intensity, X_rotation), axis=0)
# y_augmented = np.concatenate((y, y_intensity, y_rotation), axis=0)

# # Index 6000 (inclusive) to the end are AUGMENTED data samples and should
# # NOT BE INCLUDED IN THE TEST SET
# print("X_augmented shape:", X_augmented.shape)
# print("y_augmented shape:", y_augmented.shape)

In [5]:
# I was previously getting 99.6% accuracy.. there was some data leakage into the validation set

# Now I think KFold testing is successful WITH imputed data for training, validating ONLY on original data
# Making sure not to contaminate the validation data by generating augmentation
# ONLY from the training split of each fold. (the 

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

val_accuracies = []
val_losses = []

fold_i = 1
# skf.split iterates over the ORIGINAL data (X_normalized, y)
# train_index and val_index refer to indices within the original X_normalized and y
for train_index, val_index in skf.split(X_normalized, y):
    print(f'-=-=-=-= Training Fold {fold_i}/{5} -=-=-=-=')

    # using the values from the first code cell, get the the original training and validation splits for this fold
    X_train_fold_orig, X_val_fold = X_normalized[train_index], X_normalized[val_index]
    y_train_fold_orig, y_val_fold = y[train_index], y[val_index]

    print(f"fold #{fold_i}: Original training samples count: {len(X_train_fold_orig)}")
    print(f"fold #{fold_i}: Original validation samples count: {len(X_val_fold)}")

    # Generate augmented data ONLY from the original TRAINING split of this fold
    # We apply the augmentation functions *only* to X_train_fold_orig and y_train_fold_orig
    # Using proportions relative to the size of this specific training fold

    # the number of augmented training samples is floor(training split count * 0.5).. increase by increasing the last parameter
    X_intensity_fold, y_intensity_fold = vary_intensity(X_train_fold_orig, y_train_fold_orig, proportion=0.5)
    X_rotation_fold, y_rotation_fold = vary_rotation(X_train_fold_orig, y_train_fold_orig, proportion=0.5)

    print(f"fold #{fold_i}: Generated {len(X_intensity_fold)} samples by varying intensity")
    print(f"fold #{fold_i}: Generated {len(X_rotation_fold)} samples by varying rotation")

    # Now combine the original training data with the augmented data
    X_train_fold = np.concatenate((X_train_fold_orig, X_intensity_fold, X_rotation_fold), axis=0) # concatenate downward in data tensor shape (adding more sampels to the list up to down)
    y_train_fold = np.concatenate((y_train_fold_orig, y_intensity_fold, y_rotation_fold), axis=0)

    # During training, the model can get its initial "graph traversal pathways" biased if we only train with mostly original data at first
    # so we should shuffle the original and augmented samples around
    shuffle_indices = np.random.permutation(len(X_train_fold))
    X_train_fold = X_train_fold[shuffle_indices]
    y_train_fold = y_train_fold[shuffle_indices]

    print(f"fold #{fold_i} final training set size now has {X_train_fold.shape[0]} samples")

    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(512, (3, 3), activation='relu'),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.1),
        Dense(10, activation='softmax')
    ])

    # one-hot encode the labels fro the current fold
    # using the size of the COMBINED training labels
    y_train_one_hot_encoded = np.zeros((y_train_fold.shape[0], 10)) # 2d data tensors w/ each row corresponding to one data sample's one hot encoding for calculating the difference (error) between model output and labels
    # use the size of the original validation labels
    y_val_one_hot_encoded = np.zeros((y_val_fold.shape[0], 10))

    # now one hot encode the combined training labels
    for i in range(len(y_train_fold)):
        # y_train_fold[i] is shape (1,), so access element with [0]
        y_train_one_hot_encoded[i, y_train_fold[i][0]] = 1

    # one hot encode the original training labels
    for i in range(len(y_val_fold)):
         # y_val_fold[i] is shape (1,), so access element with [0]
        y_val_one_hot_encoded[i, y_val_fold[i][0]] = 1

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    print(f'-=-=-=-=-=-=- TRAINING FOR FOLD #{fold_i} -=-=-=-=-=-')
    # fit hte model to the combined (with non-augmented/original + fold-specific augmented) training data
    history = model.fit(
        X_train_fold, y_train_one_hot_encoded,
        epochs=16,
        batch_size=128,
        # validates only on the original non-imputed dataset
        validation_data=(X_val_fold, y_val_one_hot_encoded),
        verbose=0 #reduce the console output
    )

    val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_one_hot_encoded, verbose=0)
    print(f"fold #{fold_i} validation accuracy {val_accuracy}")
    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss)

    fold_i = fold_i + 1

print("\n\nCross-validation summary")
print(f"Score per fold {val_accuracies}")
print(f"Average validation accuracy {np.mean(val_accuracies)}")
print(f"Average validation loss value {np.mean(val_losses)}")

# next up: test out increasing model complexity (parameter count) while increasing the model augmentation proportion (training data size)
# in accordance with the neural scaling laws

-=-=-=-= Training Fold 1/5 -=-=-=-=
fold #1: Original training samples count: 4799
fold #1: Original validation samples count: 1200
fold #1: Generated 2399 samples by varying intensity
fold #1: Generated 2399 samples by varying rotation
fold #1 final training set size now has 9597 samples
-=-=-=-=-=-=- TRAINING FOR FOLD #1 -=-=-=-=-=-


/home/jiekaitao/anaconda3/envs/tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


fold #1 validation accuracy 0.9649999737739563
-=-=-=-= Training Fold 2/5 -=-=-=-=
fold #2: Original training samples count: 4799
fold #2: Original validation samples count: 1200
fold #2: Generated 2399 samples by varying intensity
fold #2: Generated 2399 samples by varying rotation
fold #2 final training set size now has 9597 samples
-=-=-=-=-=-=- TRAINING FOR FOLD #2 -=-=-=-=-=-
fold #2 validation accuracy 0.9449999928474426
-=-=-=-= Training Fold 3/5 -=-=-=-=
fold #3: Original training samples count: 4799
fold #3: Original validation samples count: 1200
fold #3: Generated 2399 samples by varying intensity
fold #3: Generated 2399 samples by varying rotation
fold #3 final training set size now has 9597 samples
-=-=-=-=-=-=- TRAINING FOR FOLD #3 -=-=-=-=-=-
fold #3 validation accuracy 0.9783333539962769
-=-=-=-= Training Fold 4/5 -=-=-=-=
fold #4: Original training samples count: 4799
fold #4: Original validation samples count: 1200
fold #4: Generated 2399 samples by varying intensity


In [8]:
!

Usage: sleep NUMBER[SUFFIX]...
  or:  sleep OPTION
Pause for NUMBER seconds.  SUFFIX may be 's' for seconds (the default),
'm' for minutes, 'h' for hours or 'd' for days.  NUMBER need not be an
integer.  Given two or more arguments, pause for the amount of time
specified by the sum of their values.

      --help        display this help and exit
      --version     output version information and exit

GNU coreutils online help: <https://www.gnu.org/software/coreutils/>
Full documentation <https://www.gnu.org/software/coreutils/sleep>
or available locally via: info '(coreutils) sleep invocation'


In [6]:
# ###############
# ## OLD STUFF ##
# ###############



# ###############
# ## Train test train split
# ###############


# # 96.8% accuracy w/ 500 epochs

# model = Sequential([
#     Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
#     MaxPooling2D((2, 2)), # slide a 2x2 pooling filter to reduce overfitting
#     Conv2D(64, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(128, (3, 3), activation='relu'),
#     MaxPooling2D((2, 2)),
#     Conv2D(256, (3, 3), activation='relu'),
#     Flatten(), # flatten for the input layer of the MLP
#     # mlp part
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.3),
#     Dense(64, activation='relu'),
#     Dropout(0.1),
#     Dense(10, activation='softmax') # output layer for the 10 possible classes
# ])

# y_train_one_hot_encoded = np.zeros((y_train.shape[0], 10)) # 2d data tensors w/ each row corresponding to one data sample's one hot encoding for calculating the difference (error) between model output and labels
# y_val_one_hot_encoded = np.zeros((y_val.shape[0], 10))

# for i in range(len(y_train)):
#     y_train_one_hot_encoded[i, y_train[i]] = 1
    
# for i in range(len(y_val)):
#     y_val_one_hot_encoded[i, y_val[i]] = 1

# # print(y_val_one_hot_encoded)

# y_train = y_train_one_hot_encoded
# y_val = y_val_one_hot_encoded

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()

# history = model.fit(
#     X_train_normalized, y_train,
#     epochs=16, # 16 before
#     batch_size=128,
#     validation_data=(X_val_normalized, y_val),
#     verbose=1
# )

# val_loss, val_accuracy = model.evaluate(X_val_normalized, y_val)
# print("validation accuracy:", val_accuracy)